In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("ps5.ipynb")

### Question 1
This question will continue where we left off in lecture, by using the MapReduce class to perform data processing.

In [2]:
from functools import reduce
import itertools
import gzip


class MapReduce:
    @property
    def reduce_init(self):
        # override as necessary if the init parameter needs to change
        return None

    def mapper(self, x):
        raise NotImplementedError()

    def reducer(self, accum, x):
        raise NotImplementedError()

    def postprocess(self, reduced):
        # override if necessary
        return reduced

    def run(self, iterable):
        mapped = map(self.mapper, iterable)
        reduced = reduce(self.reducer, mapped, self.reduce_init)
        processed = self.postprocess(reduced)
        return processed

(To make things more flexible, we have also added an optional `.postprocess()` method that can be used to do additional processing after the reduction step.)

Questions 1(a)-1(c) concern the Enron dataset seen in lecture:

In [3]:
def enron(n=None):
    i1 = gzip.open("email-Enron.txt.gz", "rt")
    i2 = itertools.islice(i1, 4, None)  # slice off header
    return itertools.islice(i2, n)


For each question below, implement a subclass of MapReduce such that calling `.run(enron(n))` produces the desired output. For example, if the question asked you to calcluate the total number of e-mails, your solution could be:

In [4]:
class NumEmails(MapReduce):
    @property
    def reduce_init(self):
        return 0

    def mapper(self, x):
        return 1

    def reducer(self, accum, x):
        return accum + x


NumEmails().run(enron(100))

100

**1(a)** (3 pts) Define a user's *importance* to be the number of unique people who e-mailed them (not including themself). Write a MapReduce class that returns a `collections.Counter` mapping each user ID to their importance when run.

In [5]:
class Importance(MapReduce):
    ...
    
    @property
    def reduce_init(self):
        return {}
    
    def mapper(self, line):                       
        return [int(x) for x in line.split("\t")] # [0, 1], [1, 0], [1, 2], [1, 3]
    
    def reducer(self, accumulated, x):
        sender, receiver = x # sender = 0, receiver = 1, sender = 1, receiver = 0...
        accumulated.setdefault(receiver, set()) # {0: set()} {0: {1}, 1: set()} {0: {1}, 1: {0}} {0: {1}, 1: {0, 2}} {0: {1}, 1: {0, 2, 3}} {0: {1}, 1: {0, 2, 3, 4}}
        
        if receiver != sender:
            accumulated[receiver].add(sender) # {0: {1}} {0: {1}, 1: {0}} {0: {1}, 1: {0, 2}} {0: {1}, 1: {0, 2, 3}} {0: {1}, 1: {0, 2, 3, 4}} {0: {1}, 1: {0, 2, 3, 4, 5}} {0: {1}, 1: {0, 2, 3, 4, 5, 6}}
            
        return accumulated


    def postprocess(self, reduced):
        from collections import Counter

        return Counter({k: len(v) for k, v in reduced.items()}) # translates data into counter, {0: {1}, 1: {0, 2, 3, 4, 5, 6, 7, 8, 9}} -> Counter({0: 1, 1: 9})
    
Importance().run(enron(10))

Counter({1: 1, 0: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1})

In [6]:
grader.check("q1a")

q1a results: All test cases passed!

**1(b)** (4 pts) Define a user's *forgetfulness* to be the number of times they e-mailed themself. Write a MapReduce class that returns a `Counter` that maps each user who e-mailed themself at least once to their forgetfulness score.

In [7]:
class Forgetful(MapReduce):
    
    @property
    def reduce_init(self):
        return {}
    
    def mapper(self, line):                       
        return [int(x) for x in line.split("\t")] # [0, 1], [1, 0], [1, 2], [1, 3]
    
    def reducer(self, accumulated, x):
        sender, receiver = x # sender = 0, receiver = 1, sender = 1, receiver = 0...
        accumulated.setdefault(receiver, []) # {0: set()} {0: {1}, 1: set()} {0: {1}, 1: {0}} {0: {1}, 1: {0, 2}} {0: {1}, 1: {0, 2, 3}} {0: {1}, 1: {0, 2, 3, 4}}
        
        if receiver == sender:
            accumulated[receiver].append(sender) # {0: {1}} {0: {1}, 1: {0}} {0: {1}, 1: {0, 2}} {0: {1}, 1: {0, 2, 3}} {0: {1}, 1: {0, 2, 3, 4}} {0: {1}, 1: {0, 2, 3, 4, 5}} {0: {1}, 1: {0, 2, 3, 4, 5, 6}}  
            
        return accumulated
    
    def postprocess(self, reduced):
        from collections import Counter

        return Counter({k: len(v) for k, v in reduced.items() if len(v) != 0}) # translates data into counter, {0: {1}, 1: {0, 2, 3, 4, 5, 6, 7, 8, 9}} -> Counter({0: 1, 1: 9})
    ...
    
Forgetful().run(enron(1000))

Counter({46: 3})

In [8]:
grader.check("q1b")

q1b results: All test cases passed!

**1(c)** (5 pts) Define a user's *professor score* to be the number of unique individuals who e-mailed that user and never got a response back. Write a MapReduce class that returns the a `Counter()` mapping each user with a nonzero professor score to their score.

In [40]:
class ProfessorScore(MapReduce):
    @property
    def reduce_init(self):
        return {}
    
    def mapper(self, line):                       
        return [int(x) for x in line.strip().split("\t")] # [0, 1], [1, 0], [1, 2], [1, 3]
    
    def reducer(self, accumulated, x):
        sender, receiver = x # sender = 0, receiver = 1, sender = 1, receiver = 0...
        accumulated.setdefault(receiver, set()) # {0: set()} {0: {1}, 1: set()} {0: {1}, 1: {0}} {0: {1}, 1: {0, 2}} {0: {1}, 1: {0, 2, 3}} {0: {1}, 1: {0, 2, 3, 4}}
        
        accumulated[receiver].add(sender)
        
        if sender in accumulated:
            if receiver in accumulated[sender]:
                accumulated[sender].remove(receiver)
        

        return accumulated

    def postprocess(self, reduced):
        from collections import Counter
        
        return Counter({k: len(v) for k, v in reduced.items() if len(v) != 0}) # translates data into counter, {0: {1}, 1: {0, 2, 3, 4, 5, 6, 7, 8, 9}} -> Counter({0: 1, 1: 9})
    ...

ProfessorScore().run(enron(10))

Counter({0: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1})

In [41]:
grader.check("1c")

1c results: All test cases passed!

Questions 1(d)-1(e) concern the following Facebook dataset:

In [15]:
import gzip
import itertools


def fb(n=None):
    return itertools.islice(gzip.open("fb.txt.gz", "rt"), n)

Each line of the file contains a list of integers. The first integer is a user id, and the remaining integers and the ids of all that user's Facebook friends. User 0 has a lot of friends:

In [16]:
next(fb())

'0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

Notes:
- It is *not* necessarily the case that the friend of every user is also present in the dataset.
- In this dataset, friendship is symmetric: $A$ is a friend of $B$ implies that $B$ is a friend of $A$. However, this is only indicated *once* in the data: if $A$ is a friend of $B$ and $A < B$, then $B$ appears in the friend list of $A$.

**1(d)** (5 pts) Define a *triangle* to be any set of three users $(A, B, C)$ such that they are all friends. For example, `126` is friends with both `0` and `1`, so `{0, 1, 126}` is a triangle. Write a MapReduce class that returns the set of all triangles in the dataset. Each triangle should be represented as a `frozenset` of three user ids.

In [36]:
class Triangles(MapReduce):
    """Question graded regardless of the output"""
    @property
    def reduce_init(self):
        return {}
    
    def mapper(self, line):                       
        user, *friends = map(int, line.strip().split(" "))
        keys = [tuple(sorted([user, friend])) for friend in friends] # creates set of friends: (0,1), (0,2), (0,3)...
        return {k: set(friends) for k in keys} # returns dictionary of friends, key = friends , values = common friends
    
    def reducer(self, accumulated, x):
        for key in set(sorted(x)):
            if key in accumulated:
                accumulated[key] &= x[key]
            else:
                accumulated[key] = x[key]
        return accumulated
    
    def postprocess(self, reduced):
        from collections import Counter

        return reduced # translates data into counter, {0: {1}, 1: {0, 2, 3, 4, 5, 6, 7, 8, 9}} -> Counter({0: 1, 1: 9})
    ...

In [37]:
grader.check("1d")

1d results: All test cases passed!

**1(e)** (5 pts) Social networks love to remind you how many friends you have in common with other people. Write a MapReduce class that returns, for every pair friends in the dataset, the number of friends that they have in common (not including each other). The returned data structure should be a `dict` whose keys are `frozenset()`s containing two userids, and values are integers giving the number of friends in common.

In [38]:
class CommonFriends(MapReduce):
    """Question graded regardless of the output"""
    @property
    def reduce_init(self):
        return {}
    
    def mapper(self, line):                       
        user, *friends = map(int, line.strip().split(" "))
        keys = [tuple(sorted([user, friend])) for friend in friends] # creates set of friends: (0,1), (0,2), (0,3)...
        return {k: set(friends) for k in keys} # returns dictionary of friends, key = friends , values = common friends
    
    def reducer(self, accumulated, x):
        for key in set(sorted(x)):
            if key in accumulated:
                accumulated[key] &= x[key]
            else:
                accumulated[key] = x[key]
        return accumulated
    
    def postprocess(self, reduced):
        from collections import Counter

        return reduced # translates data into counter, {0: {1}, 1: {0, 2, 3, 4, 5, 6, 7, 8, 9}} -> Counter({0: 1, 1: 9})
    ...

In [39]:
grader.check("1e")

1e results: All test cases passed!

## Question 2: Conway's game of life

[Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) 
is a game devised by the late mathematician John Conway.

The game is played on an $m$-by-$n$ board, which we
will represent as an $m$-by-$n$ Numpy array. 
Each cell of the board (i.e., entry of our matrix), is
either alive (which we will represent as a $1$) or dead (which we will
represent as a $0$). 

The game proceeds in steps. At each step of the game, the board evolves according to the following rules:

-   A live cell with fewer than two live neighbors becomes a dead cell.

-   A live cell with more than three live neighbors becomes a dead cell.

-   A live cell with two or three live neighbors remains alive.

-   A dead cell with *exactly* three live neighbors becomes alive.

-   All other dead cells remain dead.

The neighbors of a cell are the 8 cells adjacent to it, i.e., left,
right, above, below, upper-left, lower-left, upper-right and
lower-right. Thus, in a $5\times 5$ game of life, the neighors of the middle cell are (shown in black):
```
⬜⬜⬜⬜⬜
⬜⬛⬛⬛⬜
⬜⬛⬜⬛⬜
⬜⬛⬛⬛⬜
⬜⬜⬜⬜⬜
```
For cells that are on the boundary, we will follow the convention that the board is 
*toroidal*, meaning that it wraps around the other side. So the neighbors of the top-left most
square are (again shown in black):
```
⬜⬛⬜⬜⬛
⬛⬛⬜⬜⬛
⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
⬛⬛⬜⬜⬛
```
In matrix notation, this means that the top-left neighbor of cell $(i,j)$ is $(i-1 \mod m, j-1 \mod n)$, etc.

Write a class `GameOfLife` that plays this game:

In [453]:
import numpy as np


class GameOfLife:
    def __init__(self, array):
        self.array = array
        for li in array:
            for num in li:
                if not (num==1 or num ==0):
                    raise ValueError('Input has to be 0 and 1.')
        self.check = []
    ...                

    def __str__(self):
        matrix_to_str = str(self.array)
        to_print = []
        for string in matrix_to_str:
            new_string = string.replace("0", "\u2b1c")
            new_string = new_string.replace("1", "\u2B1B")
            to_print.append(new_string)
            
        return ''.join(to_print).replace('[','').replace(']','').replace(' ','')
        

    def change_stat(self, arr):

        # Create new array - next phase in the game
        cols = np.shape(arr)[1]
        rows = np.shape(arr)[0]
        next_arr = np.zeros(shape = (rows, cols), dtype=int)
        
        for row in range(rows):       # row iteration
            for col in range(cols):   # column itetarion
            # iterate surrounding neighbors
                _total = 0
                for i in range(-1, 2):
                    for j in range(-1, 2):     
                        if not (j == 0 and i == 0):
                            _total = _total + arr[((row + i) % rows)][((col + j) % cols)]
                 
                if _total == 3 and arr[row][col] == 0:
                    next_arr[row][col] = 1
                elif _total > 3 or _total < 2:
                    next_arr[row][col] = 0
                else:
                    next_arr[row][col] = arr[row][col]
            
        # Terminate the game if new games are the same as the old ones
        self.check.append(next_arr)
        
        if len(self.check) > 1:
            if not np.array_equal(self.check[-1], self.check[-2]): 
                self.array = next_arr
                return next_arr
            else:
                raise StopIteration
        else:
            self.array = next_arr
            return next_arr
    
    def __next__(self):
        return self.change_stat(self.array)
    
    def __iter__(self):
        return self
            
    ...
    
...

Ellipsis

Instances of the class should behave as follows:

**2(a)** (2 pts) The constructor of `GameOfLife` should accept a single argument, which is a two-dimensional Numpy integer array, and perform validation. A starting board is valid if it contains only zero and ones. (You may assume that the input is a 2-d Numpy integer array, but your constructor should check the part about zeros and ones.)

In [454]:
grader.check("2a")

2a results: All test cases passed!

**2(b)** (3 pts) Instances of `GameOfLife` should return a string representation of the current state of the game:
```
>>> I = np.eye(5, dtype=int)  # 5x5 identity matrix
>>> g = GameOfLife(I)
>>> print(g)
⬛⬜⬜⬜⬜
⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬛
```
In the string representation, use the Unicode characters "⬛" to denote a live cell and "⬜" to denote a dead cell. In Python, these can be inputted as:

In [455]:
"\u2b1c", "\u2B1B"

('⬜', '⬛')

In [456]:
grader.check("2b")

2b results: All test cases passed!

**2(c)** (10 pts) `GameOfLife` instances should be iterable. Calling `next` on the instance should return the next state of the game, represented as a Numpy array, which is determined by applying the rules stated above. If the game terminates, meaning that the board remains the same from one turn to the next, then the iterator terminates (by raising a `StopIteration`).

Here is an example of the game played using a $5 \times 5$ grid and a pattern that oscillates back and forth:

```
>>> blinker = np.zeros((5, 5), dtype=int) 
>>> blinker[1:4, 2] = 1
>>> g = GameOfLife(blinker)
>>> print(g)
⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜
>>> next(g)
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 1 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
>>> print(g)
⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬛⬛⬛⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
>>> next(g)
[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]
>>> print(g)
⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜
```

*Hint*: the main challenge to this exercise is in dealing with the toroidal matrix. To avoid having to consider special cases, use the fact that the neighbors of cell $(i,j)$ are $(i\pm 1 \mod m, j\pm 1 \mod n)$ where $a \mod b$ denotes the modulus (i.e. `a % b` in Python.)

In [457]:
grader.check("2c")

2c results: All test cases passed!

**2(d)** (just for fun) The following code will print out a Game of Life as it runs:

In [376]:
import ipywidgets as widgets
import time
import itertools

out = widgets.Output()


def play_gol(game, stop=None, wait=0.5):
    "play game of life for stop steps and show the output, waiting wait between each frame."
    for step in itertools.islice(game, stop):
        with out:
            print(game)
            out.clear_output(wait=True)
        time.sleep(wait)


out

Output()

Example:

In [377]:
glider = np.zeros((10, 10), dtype=int)
glider[5, 3:6] = 1
glider[4, 5] = 1
glider[3, 4] = 1
g = GameOfLife(glider)
# uncomment next line to view game
# play_gol(g, 10, 0.1)  

Try running it with your own invented initial state. Can you make anything interesting happen? Some people have developed extremely elaborate games, for example:

In [378]:
tr = str.maketrans(".X", "01")
gosper = np.array(
    [list(map(int, row.strip().translate(tr))) for row in open("gosper-glider-gun.txt")]
)
# uncomment next line to view game
# play_gol(GameOfLife(gosper), 100, wait=0.01)

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [379]:
grader.check_all()

1c results: All test cases passed!

1d results: All test cases passed!

1e results: All test cases passed!

2a results: All test cases passed!

2b results: All test cases passed!

2c results: All test cases passed!

q1a results: All test cases passed!

q1b results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)